In [26]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import log_loss

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [27]:
df = pd.read_csv("../MachinehackGlassQualityPrediction/Glass_Quality_Participants_Data/Train.csv")
tf = pd.read_csv("../MachinehackGlassQualityPrediction/Glass_Quality_Participants_Data/Test.csv")

sf = pd.read_excel("../MachinehackGlassQualityPrediction/Glass_Quality_Participants_Data/Sample_Submission.xlsx")


In [28]:
df['xmax'] = np.sqrt(df['xmax'])
df['ymax'] = np.sqrt(df['ymax'])
df['xmin'] = np.sqrt(df['xmin'])
df['ymin'] = np.sqrt(df['ymin'])

df['max_luminosity'] = np.log(df['max_luminosity'])
df['pixel_area'] = np.sqrt(df['pixel_area'])
df['log_area'] = np.sqrt(df['log_area'])

tf['xmax'] = np.sqrt(tf['xmax'])
tf['ymax'] = np.sqrt(tf['ymax'])
tf['xmin'] = np.sqrt(tf['xmin'])
tf['ymin'] = np.sqrt(tf['ymin'])

tf['max_luminosity'] = np.log(tf['max_luminosity'])
tf['pixel_area'] = np.sqrt(tf['pixel_area'])
tf['log_area'] = np.sqrt(tf['log_area'])



In [29]:
#df =df.drop('pixel_area',axis=1)

# tf =tf.drop('log_area',axis=1)

In [30]:
# adding  interaction new features 
import itertools
interactions = pd.DataFrame(index=df.index)
interactions

""
0
1
2
3
4
...
1353
1354
1355
1356


In [31]:
features =['max_luminosity','thickness','pixel_area']
for col1 ,col2 in  itertools.combinations(features,2):
    
    newcolname = col1 + "_" + col2 
    new_values = df[col1].map(str) + "_" + df[col2].map(str)
    interactions[newcolname] = new_values


In [32]:
interactions['max_luminosity_thickness']

0       5.0106352940962555_36
1        4.897839799950911_55
2         4.90527477843843_65
3        4.812184355372417_35
4        4.927253685157205_69
                ...          
1353     4.890349128221754_37
1354     4.976733742420574_35
1355     4.976733742420574_47
1356     4.836281906951478_84
1357     5.093750200806762_40
Name: max_luminosity_thickness, Length: 1358, dtype: object

In [33]:
#df =df.join(interactions)

In [34]:
interactionstest = pd.DataFrame(index=tf.index)
interactionstest

""
0
1
2
3
4
...
578
579
580
581


In [35]:
features =['max_luminosity','thickness','pixel_area']
for col1 ,col2 in  itertools.combinations(features,2):
    
    newcolname = col1 + "_" + col2 
    new_values = tf[col1].map(str) + "_" + tf[col2].map(str)
    interactionstest[newcolname] = new_values


In [36]:
#tf =tf.join(interactionstest)

In [37]:
tf

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5
0,1,0,4.812184,65,5.744563,7.348469,1283.136392,1283.313290,25.139610,25.147400,0,1,0,0,0
1,0,1,5.062595,99,11.180340,11.489125,435.745338,435.349285,20.518285,20.540901,0,0,0,0,0
2,0,1,4.787492,35,8.000000,8.660254,3603.730428,3603.728902,16.492423,16.498855,1,0,0,0,0
3,0,1,4.867534,39,7.549834,13.304135,556.447662,557.515919,57.549978,57.552677,0,0,1,0,0
4,1,0,4.955827,75,4.795832,8.888194,2316.960725,2316.779446,29.359837,29.368336,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,1,0,5.030438,55,38.457769,38.652296,1117.884609,1117.854194,14.071247,14.101301,0,0,0,0,0
579,0,1,4.990433,43,10.148892,12.688578,396.981108,395.682954,51.990384,51.999862,0,0,1,0,0
580,0,1,4.948760,44,1.000000,19.697716,714.102234,715.395695,145.585027,145.586494,0,0,1,0,0
581,0,1,5.356586,150,20.346990,21.236761,371.070074,371.502355,95.754895,95.756395,1,0,0,0,0


In [38]:
df.head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,class
0,0,1,5.010635,36,12.000000,13.114877,973.254848,973.309817,20.952327,20.954696,0,0,1,0,0,1
1,1,0,4.897840,55,33.823069,33.941125,1542.419528,1542.602995,18.138357,18.144024,1,0,0,0,0,1
2,1,0,4.905275,65,30.822070,31.208973,1019.039744,1018.211177,17.320508,17.323989,0,0,0,0,0,2
3,0,1,4.812184,35,6.403124,14.832397,1305.978560,1305.987749,82.480301,82.485021,0,0,1,0,0,1
4,1,0,4.927254,69,21.494185,21.587033,1043.131823,1042.391001,15.842980,15.855660,0,0,0,0,0,2


In [39]:
enc = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
enc.fit(np.array(df['max_luminosity']).reshape(-1,1))
df['max_luminosity_binned'] = enc.transform(np.array(df['max_luminosity']).reshape(-1,1))
tf['max_luminosity_binned'] = enc.transform(np.array(tf['max_luminosity']).reshape(-1,1))


In [40]:
enc = KBinsDiscretizer(n_bins=7, encode='ordinal', strategy='kmeans')
enc.fit(np.array(df['thickness']).reshape(-1,1))
df['thickness_binned'] = enc.transform(np.array(df['thickness']).reshape(-1,1))
tf['thickness_binned'] = enc.transform(np.array(tf['thickness']).reshape(-1,1))


In [41]:
df.to_csv('../MachinehackGlassQualityPrediction/LgbmcTrain.csv' , index=False)
tf.to_csv('../MachinehackGlassQualityPrediction/LgbmcTest.csv', index= False)


In [17]:

def targetencoding(train,test,y_train):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = ['max_luminosity_thickness','max_luminosity_pixel_area','thickness_pixel_area']
    print(f'targest emcoding for features {cat_features}')
    target_enc = ce.TargetEncoder(cols=cat_features)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_target'))

    traintrgtenc = traintrgtenc.drop(cat_features, axis =1)
    testtrgtenc = testtrgtenc.drop(cat_features, axis =1)

    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc

In [18]:
#df,tf = targetencoding(df,tf,df['class'])

In [19]:
df.head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,class,max_luminosity_binned,thickness_binned
0,0,1,5.010635,36,12.000000,13.114877,973.254848,973.309817,20.952327,20.954696,0,0,1,0,0,1,3.0,0.0
1,1,0,4.897840,55,33.823069,33.941125,1542.419528,1542.602995,18.138357,18.144024,1,0,0,0,0,1,2.0,0.0
2,1,0,4.905275,65,30.822070,31.208973,1019.039744,1018.211177,17.320508,17.323989,0,0,0,0,0,2,2.0,1.0
3,0,1,4.812184,35,6.403124,14.832397,1305.978560,1305.987749,82.480301,82.485021,0,0,1,0,0,1,1.0,0.0
4,1,0,4.927254,69,21.494185,21.587033,1043.131823,1042.391001,15.842980,15.855660,0,0,0,0,0,2,2.0,1.0


In [20]:
y = df['class']

# To keep things simple, we'll use only numerical predictors
X = df.drop(['class'], axis=1)

#X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.70, test_size=0.30,random_state=0)

In [21]:
model =LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators = 400, n_jobs=-1, num_leaves=15, objective='binary',
               random_state=2, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [22]:
# from sklearn.model_selection import cross_validate, cross_val_predict

In [23]:
# score = cross_validate(model, X, y,
#                            scoring=['neg_log_loss'],
#                            n_jobs=-1, verbose=0)



In [24]:
# kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
# y_pred = cross_val_predict(model, X, y, n_jobs=-1,method ='predict_proba',cv=kf ,verbose=0)

In [25]:
model.fit(X,y)

pred = model.predict_proba(tf)
prediction = pd.DataFrame(data =pred, columns=sf.columns)
prediction.to_excel('../MachinehackGlassQualityPrediction/LGBMCnofe.xlsx')
